In [193]:
import pickle
import json
import random
import tensorflow
import tflearn
import numpy
import nltk

In [195]:
with open("intents.json", 'rb') as file:
    data = json.load(file)

In [196]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'Hello',
    'Hola',
    'Hey',
    'Hi',
    'Hello',
    'Good morning',
    'Good Evening'],
   'responses': ['Hello, Welcome to Our IT Chatbot',
    'Good to see you, Welcome to our IT Chatbot',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time',
    'Bye Chatbot'],
   'responses': ['See you!',
    'Have a nice day',
    'Bye! Come back again soon.',
    'Happy to help'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help! Any other issues?',
    'Any time! Any other issues I can help with?',
    'My pleasure! Any other Issues I can help with?'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': ['q', 'random'],
   'responses': ["Sorry, 

In [215]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [216]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [217]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

In [218]:
labels = sorted(labels)

In [219]:
training = []
output = []

In [220]:
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

In [222]:
training = numpy.array(training)
output = numpy.array(output)

In [298]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
#net = tflearn.lstm(net, 128, dropout=0.8, return_seq=True)
#net = tflearn.lstm(net, 128, dropout=0.8, return_seq=True)
#net = tflearn.lstm(net, 128, dropout=0.8, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=0)
history = model.fit(training, output, n_epoch=300, batch_size=32, show_metric=True, run_id='intents')
#model.save("model.tflearn")

---------------------------------
Run id: intents
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 88
Validation samples: 0
--


In [285]:
history

In [286]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [287]:
def chat():
    print("Comece a conversar com o bot (digite quit para parar)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            print(random.choice(responses))
        else:
            print("I didn't get that, try again. ")

In [288]:
chat()

Comece a conversar com o bot (digite quit para parar)!
You: USB
I didn't get that, try again. 
You: Password Reset
I didn't get that, try again. 
You: Reset Password
I didn't get that, try again. 
You: Hi
Hi there, how can I help?
You: Password Reset
I didn't get that, try again. 
You: How to reset my password?
Confirm your email address
You: flasile@gmail.com
The reset pin has been sent on your registered mobile number
You: Vague
See you!
You: Nothing works
Hey, I cant seems that you have not listed you Issue here.
 I can help you out with the following issues: 
 1)Password Reset
 2) Trouble-Shooting issues
 3) Virus Issues 
 4)Printing Issues and many more IT issues
You: Virus issues
A detailed step by step guide to remove the virus from the computer has been provided on the following link: 
https://www.easeus.com/file-recovery/remove-virus-without-antivirus.html 
and
https://www.pcworld.com/article/243818/how-to-remove-malware-from-your-windows-pc.html
If the issues are still there,

KeyboardInterrupt: Interrupted by user